In [2]:
import pandas as pd
pd.options.plotting.backend = "plotly"
import plotly.express as px

import numpy as np

In [3]:
df_prod = pd.read_csv("./data/world_metal_primary_production_2017_2021.csv")
df_acv = pd.read_excel("./data/LCA_metals_2008_PLOS_agregated.xlsx")
metals_name2symbol = pd.read_csv("./metal_name2symbol.csv")

df_prod = pd.merge(df_prod, metals_name2symbol, how="left")

In [4]:
df_acv.head()

,Z,Material,2008 Global Production [kg],CED [MJ/kg],CED - 2.5% [MJ/kg],CED - 97.5% [MJ/kg],Global Cumulative Energy \nDemand (2008) [PJ/yr],GWP 100a\n[kg CO2-eq/ kg],GWP - 2.5% [kg CO2-eq/ kg],GWP - 97.5% [kg CO2-eq/ kg],Global CO2 Emissions (2008) \n[kg CO2-eq/yr]
0,2,He,2.975E+07,67.5,41.4,103.0,2.0,0.9,0.6,1.4,2.77E+07
1,3,Li,1.410E+08,125.0,94.0,161.0,6.0,7.1,5.7,8.9,4.33E+08
2,4,Be,4.000E+05,1720.0,1020.0,3440.0,0.7,122.0,79.9,191.0,5.02E+07
3,5,B,5.277E+09,27.3,15.5,46.4,144.0,1.5,0.9,2.4,7.75E+09
4,12,Mg,2.146E+10,18.8,14.1,24.4,61.1,5.4,2.0,14.4,2.26E+10


In [5]:
df_prod.head()

,Type,Commodity,2017,2018,2019,2020,2021,Material
0,Metal,Fer,1511758884,1524574245,1551335349,1524139484,1635056040,Fe
1,Metal,Chrome,14142863,15188744,15779216,11967592,14676208,Cr
2,Metal,Cobalt,138692,158317,124518,130056,134476,Co
3,Metal,Manganèse,18758889,19609756,21164890,19623184,21219335,Mn
4,Metal,Molybdène,285378,270100,277180,284890,267724,Mo


In [12]:
# Consommation énergétique et émissions totales pour les métaux

df = pd.merge(df_prod, df_acv, how="left", on="Material")
df = df[~df["CED [MJ/kg]"].isna()]
df["Global Cumulative Energy Demand (2021) [EJ/yr]"] = df["2021"]*1e3 * df["CED [MJ/kg]"] * 1e6 * 1e-18
df["Global CO2 Emissions (2021) [MtCO2eq/yr]"] = df["2021"]*1e3 * df["GWP 100a\n[kg CO2-eq/ kg]"] * 1e-9

In [13]:
# Total 
df[["Global Cumulative Energy Demand (2021) [EJ/yr]","Global CO2 Emissions (2021) [MtCO2eq/yr]"]].sum()

Global Cumulative Energy Demand (2021) [EJ/yr]      51.984358
Global CO2 Emissions (2021) [MtCO2eq/yr]          3319.959122
dtype: float64

In [14]:
fig = px.treemap(df, path=["Material"], values='Global CO2 Emissions (2021) [MtCO2eq/yr]',custom_data=['Global CO2 Emissions (2021) [MtCO2eq/yr]'])
fig.update_traces(texttemplate='%{label} (%{customdata:.0f} MtCO2eq)', textposition='top left')

fig.show()
fig.write_image("./figures/emissions_2021_tous_metaux.pdf",width=1000,height=500)

/home/sbrisson/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:1637: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all_trees = df_all_trees.append(df_tree, ignore_index=True)


In [63]:
fig = px.treemap(df, path=["Material"], values='Global Cumulative Energy Demand (2021) [EJ/yr]',custom_data=['Global Cumulative Energy Demand (2021) [TWh/yr]'])
fig.update_traces(texttemplate='%{label} (%{customdata:.0f} TWh)', textposition='top left')

fig.show()
fig.write_image("./figures/consommation_2021_tous_metaux.pdf",width=1000,height=500)

/home/sbrisson/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [20]:
# Scénarios de demande 

df_iea = pd.read_csv("./data/IEA_estimation_production.csv")
# df_iea = df_iea[df_iea["Scénario"] == "SDS"] 
df_iea = pd.merge(df_iea, df, how="left",on="Material")

df_iea["2040"] = df_iea["Demande 2040 (rel. à 2021)"] * df_iea["2021"]

df_iea = df_iea[~df_iea["CED [MJ/kg]"].isna()]
df_iea["Global Cumulative Energy Demand (2040) [TWh/yr]"] = df_iea["2040"]*1e3 * df_iea["CED [MJ/kg]"] * 1e-6 / 3600
df_iea["Global CO2 Emissions (2040) [MtCO2eq/yr]"] = df_iea["2040"]*1e3 * df_iea["GWP 100a\n[kg CO2-eq/ kg]"] * 1e-9
df_iea["Global Cumulative Energy Demand (2021) [TWh/yr]"] = df_iea["2021"]*1e3 * df_iea["CED [MJ/kg]"] * 1e-6 / 3600
df_iea["Global CO2 Emissions (2021) [MtCO2eq/yr]"] = df_iea["2021"]*1e3 * df_iea["GWP 100a\n[kg CO2-eq/ kg]"] * 1e-9


df_iea
# fig = px.treemap(df_iea, path=["Material"], values='Global CO2 Emissions (2040) [MtCO2eq/yr]',custom_data=['Global CO2 Emissions (2040) [MtCO2eq/yr]'])
# fig.update_traces(texttemplate='%{label} (%{customdata:.0f} MtCO2eq/an)', textposition='top left')

# fig.show()
# fig.write_image("./figures/consommation_2040_IEA_SDS.png",width=1000,height=500)

df3 = df_iea[["Commodity","Scénario","Demande 2040 (rel. à 2021)","Global CO2 Emissions (2040) [MtCO2eq/yr]","Global CO2 Emissions (2021) [MtCO2eq/yr]"]].set_index(["Scénario","Commodity"])

df3["Global CO2 Emissions (2040) [MtCO2eq/yr]"] = df3["Global CO2 Emissions (2040) [MtCO2eq/yr]"].apply(lambda x: round(x, 2 - int(np.floor(np.log10(abs(x))))))
df3["Global CO2 Emissions (2021) [MtCO2eq/yr]"] = df3["Global CO2 Emissions (2021) [MtCO2eq/yr]"].apply(lambda x: round(x, 2 - int(np.floor(np.log10(abs(x))))))


df3.to_csv("IEA_scenarios_emissions.csv")


In [9]:
# All metals 
fig = df_prod[df_prod["Type"]=="Metal"].sort_values("2021", ascending=False)[["Commodity","2021"]].plot.bar(x="Commodity",y="2021")

fig.update_yaxes(
    type="log",
    dtick = 1,
    tickvals=[10**n for n in range(2,10)],
    ticktext=["100t","1kt","10kt","100kt","1Mt","10Mt","100Mt","1Gt"],
    exponentformat = 'power'
    )
fig.update_xaxes(tickangle= -45)  
fig.update_layout(
    xaxis_title = "",
    yaxis_title = "Production primaire (2021)",
    template = "plotly_white",
    )
